In [1]:
!pip install stellargraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 435 kB 6.8 MB/s 


In [2]:
import os
import csv
import pdb
import math
import time 
import pickle
import pickle
import random
import warnings
import datetime 
import collections
import numpy as np 
import pandas as pd  
import xgboost as xgb
import networkx as nx
from tqdm.notebook import tqdm
import matplotlib.pylab as plt 
from matplotlib import rcParams   
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
import time

# Algo Import
import tensorflow as tf
from tensorflow import keras
from stellargraph.mapper import (
    CorruptedGenerator,
    FullBatchNodeGenerator,
    GraphSAGENodeGenerator,
    HinSAGENodeGenerator,
    ClusterNodeGenerator,
)
from stellargraph import StellarGraph
from stellargraph.layer import GCN, DeepGraphInfomax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Model

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
data_path = "gdrive/My Drive/GitHub Link Prediction/"

In [5]:
df_train = pd.read_csv(data_path+"gh_pos_train_91.csv").drop(columns = ['Unnamed: 0'])

In [6]:
df_train.head()

,src,dest,link
0,6639,22064,1
1,2040,26233,1
2,23636,36468,1
3,9723,17816,1
4,5068,16240,1


In [7]:
import tensorflow

In [8]:
from tensorflow import keras

In [9]:
from stellargraph.core import StellarGraph
from stellargraph.core import StellarDiGraph
from stellargraph.mapper import AdjacencyPowerGenerator
from stellargraph.layer import WatchYourStep
from stellargraph.losses import graph_log_likelihood
from stellargraph.utils import plot_history

from tensorflow.keras import optimizers, Model, layers, regularizers
import tensorflow as tf

In [10]:
from stellargraph.data import EdgeSplitter, UniformRandomWalk, UnsupervisedSampler
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

In [11]:
train_graph = nx.from_pandas_edgelist(df_train, source='src', target='dest', create_using=nx.DiGraph())

In [24]:
degrees = nx.degree(train_graph)
degrees = dict(degrees)

features = {}
for key, val in degrees.items():
  features[key] = [val]

pr = nx.pagerank(train_graph, alpha=0.85)
pr = dict(pr)
for key, val in pr.items():
  features[key].append(val)

katz = nx.katz_centrality(train_graph,alpha=0.005,beta=1)
katz = dict(katz)
for key, val in katz.items():
  features[key].append(val)

In [25]:
nx.set_node_attributes(train_graph, features, "features")

In [26]:
G = StellarGraph.from_networkx(train_graph, node_features="features")
print(G.info())

StellarDiGraph: Directed multigraph
 Nodes: 37700, Edges: 255678

 Node types:
  default: [37700]
    Features: float32 vector, length 3
    Edge types: default-default->default

 Edge types:
    default-default->default: [255678]
        Weights: all 1 (default)
        Features: none


In [27]:
fullbatch_generator = FullBatchNodeGenerator(G, sparse=True)
gcn_model = GCN(layer_sizes=[64], activations=['relu'], generator=fullbatch_generator)

corrupted_generator = CorruptedGenerator(fullbatch_generator)
gen = corrupted_generator.flow(G.nodes())

Using GCN (local pooling) filters...


In [28]:
infomax = DeepGraphInfomax(gcn_model, corrupted_generator)
x_in, x_out = infomax.in_out_tensors()

model = Model(inputs=x_in, outputs=x_out)
model.compile(loss=tf.nn.sigmoid_cross_entropy_with_logits, optimizer=Adam(lr=1e-3))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [29]:
batch_size=128
epochs=20
es = EarlyStopping(monitor="loss", min_delta=0, patience=4, restore_best_weights=True,verbose=1)

In [30]:
history = model.fit(gen, epochs=epochs, verbose=1, callbacks=[es])
model_file = open(data_path+"gcn_model.h5", "wb")
pickle.dump(model, model_file)
model_file.close()

Epoch 1/20
1/1 [==============================] - 4s 4s/step - loss: 1.5429
Epoch 2/20
1/1 [==============================] - 0s 61ms/step - loss: 0.7372
Epoch 3/20
1/1 [==============================] - 0s 59ms/step - loss: 2.4091
Epoch 4/20
1/1 [==============================] - 0s 59ms/step - loss: 1.0330
Epoch 5/20
1/1 [==============================] - 0s 62ms/step - loss: 0.7379
Epoch 6/20
1/1 [==============================] - 0s 65ms/step - loss: 1.0578
Epoch 6: early stopping
INFO:tensorflow:Assets written to: ram://749890d7-3c7e-422d-9c17-52086d2259d4/assets


In [31]:
x_emb_in, x_emb_out = gcn_model.in_out_tensors()

x_out = tf.squeeze(x_emb_out, axis=0)
emb_model = Model(inputs=x_emb_in, outputs=x_out)

In [32]:
train_gen = fullbatch_generator.flow(G.nodes())
emb = emb_model.predict(train_gen)

In [33]:
emb_dict = {}
for i, node in enumerate(list(G.nodes())):
  emb_dict[node] = np.array(emb[i].reshape(1,64))

In [35]:
combined_tr = pd.read_csv(data_path+"/gh_train_91.csv", index_col = 0)
print(combined_tr.shape)
combined_tr.head()

(516678, 3)


,src,dest,link
132913,10080,28269,1
97601,7104,31695,1
127442,21330,20345,0
189136,13326,30293,0
61075,13218,37521,0


In [36]:
combined_te = pd.read_csv(data_path+"/gh_test_91.csv", index_col = 0)
print(combined_te.shape)
combined_te.head()

(62325, 3)


,src,dest,link
222440,28906,32893,0
48306,35757,23700,0
265591,26731,28777,1
132547,5482,11624,0
124965,24379,459,0


In [46]:
# get training embeddings
train_emb = []
for i, row in tqdm(combined_tr.iterrows()):
  comb_emb = (emb_dict[row['src']]) * (emb_dict[row['dest']])
  train_emb.append(comb_emb[0])
train_emb = np.array(train_emb)
print(train_emb.shape)
train_y = combined_tr['link'].values
print(train_y.shape)

0it [00:00, ?it/s]

(516678, 64)
(516678,)


In [47]:
# get test embeddings
test_emb = []
for i, row in tqdm(combined_te.iterrows()):
  comb_emb = (emb_dict[row['src']]) * (emb_dict[row['dest']])
  test_emb.append(comb_emb[0])
print(type(test_emb))
test_emb = np.array(test_emb)
print(test_emb.shape)
test_y = combined_te['link'].values 
print(test_y.shape)

0it [00:00, ?it/s]

<class 'list'>
(62325, 64)
(62325,)


In [ ]:
# train_emb_sample, _, train_y_sample, _ = train_test_split(train_emb, train_y, stratify = train_y, test_size = 0.5, random_state = 42)

In [48]:
from lightgbm import LGBMClassifier
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.7743980033332513 test Score 0.7801393505497315


In [49]:
#reading
from pandas import read_hdf
df_final_train = read_hdf(data_path + 'gh_fea_sample/gh_storage_sample_stage4.h5', 'train_df',mode='r')
df_final_test = read_hdf(data_path + 'gh_fea_sample/gh_storage_sample_stage4.h5', 'test_df',mode='r')

In [50]:
df_final_train.columns

Index(['src', 'dest', 'link', 'jaccard_followers', 'jaccard_followees',
       'cosine_followers', 'cosine_followees', 'num_followers_s',
       'num_followers_d', 'num_followees_s', 'num_followees_d',
       'inter_followers', 'inter_followees', 'adar_index', 'follows_back',
       'same_comp', 'shortest_path', 'weight_in', 'weight_out', 'weight_f1',
       'weight_f2', 'weight_f3', 'weight_f4', 'page_rank_s', 'page_rank_d',
       'katz_s', 'katz_d', 'hubs_s', 'hubs_d', 'authorities_s',
       'authorities_d', 'svd_u_s_1', 'svd_u_s_2', 'svd_u_s_3', 'svd_u_s_4',
       'svd_u_s_5', 'svd_u_s_6', 'svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3',
       'svd_u_d_4', 'svd_u_d_5', 'svd_u_d_6', 'svd_v_s_1', 'svd_v_s_2',
       'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6', 'svd_v_d_1',
       'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5', 'svd_v_d_6',
       'svd_dot_u', 'svd_dot_v', 'pref_attach_er', 'pref_attach_ee'],
      dtype='object')

In [51]:
df_final_train.shape

(516678, 59)

In [52]:
y_train = df_final_train.link
y_test = df_final_test.link

In [53]:
df_final_train.drop(['src', 'dest','link'],axis=1,inplace=True)
df_final_test.drop(['src', 'dest','link'],axis=1,inplace=True)

In [54]:
# GCN only
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.7807843001141312 test Score 0.7841664810513252


In [55]:
# GCN only
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.7778876408174629 test Score 0.7788079470198676


In [56]:
# GCN only
from lightgbm import LGBMClassifier
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.7743980033332513 test Score 0.7801393505497315


In [57]:
# Heuristic only
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9650688498879325 test Score 0.8361208524413272


In [58]:
# Heuristic only
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9586408046760307 test Score 0.8543679731353603


In [59]:
# Heuristic only
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.965836146176809 test Score 0.8412357286769645


In [60]:
df_final_train = np.hstack((df_final_train, train_emb))
print(df_final_train.shape)
df_final_test = np.hstack((df_final_test, test_emb))
print(df_final_test.shape)

(516678, 120)
(62325, 120)


In [61]:
# Heuristic + GCN
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9652578202584348 test Score 0.8356037203963471


In [62]:
# Heuristic + GCN
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9564584271757195 test Score 0.8629880795563578


In [63]:
# Heuristic + GCN
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9658779440539523 test Score 0.8416493183802296
